In [ ]:
from data_cleaning_functions import cleaning_pipeline
from information import create_info_object, machine, kpi, features, identity, ML_algorithms_config
from machine_learning_functions import AnomalyDetector
from machine_learning_functions import ADWIN_drift
from machine_learning_functions import tdnn_forecasting_training
from machine_learning_functions import tdnn_forecasting_prediction
from feature_engineering_functions import feature_engineering_pipeline

In [ ]:
def get_datapoint():
    # In some manner receives data point as a dictionary of form
    ''' datapoint={timestamp: datetime, 
        isset_id: string,
        name: string,
        kpi: string,
        operation: string,
        sum: float
        avg: float,
        min: float,
        max: float,
        var: float}'''
    #for example

    datapoint = {
    'timestamp': 'timepoint',
    'isset_id': 'ast-yhccl1zjue2t',
    'name': 'metal_cutting',
    'kpi': 'time',
    'operation': 'working',
    'sum': float, 
    'avg': float,
    'min': float,
    'max': float,
    'var': float}
    
    return datapoint

In [ ]:
def get_historical_data(machine_name, asset_id, kpi, operation, timestap_start, timestamp_end):
    # In some manner receives data frame filtered from the database in format dataframe
    #Maybe we can define that if we give timestap_start = None, timestamp_end = None,
    #they have to return us x values in the past starting from the last stored point

    return filtered_dataframe 

In [ ]:
def send_alert(anomaly_identity):
    
    # In some manner calls the alert function and sends the identity

    return None

In [ ]:
def store_datapoint(new_datapoint):
    
    # In some manner gives the new_datapoint dictionary to the database, so they can store it

    return None

In [ ]:
def get_request(machine_name, asset_id, kpi, operation, timestap_start, timestamp_end, transformation, forecasting):
    

    transformation_config = {
        'make_stationary': False,  # Default: False
        'detrend': False,          # Default: False
        'deseasonalize': False,    # Default: False
        'get_residuals': False,    # Default: False
        'scaler': False             # Default: False
    }

    if forecasting:
        transformation_config['make_stationary'] = True
        transformation_config['scaler'] = True
        
        historical_data = get_historical_data(machine_name, asset_id, kpi, operation, None, None) ## CONNECTION WITH API

        transformed_data = feature_engineering_pipeline(historical_data, transformation_config)

        forecasting_model_info = im.get_model_forecast(historical_data[-1])

        forecasting_model = forecasting_model_info[0]
        forecasting_params = forecasting_model_info[1]
        forecasting_stats = forecasting_model_info[2]
        predictions = tdnn_forecasting_prediction(forecasting_model, forecasting_params['tau'], transformed_data, timestap_start, timestamp_end, forecasting_stats)

        return predictions
    
    else:

        historical_data = get_historical_data(machine_name, asset_id, kpi, operation, timestap_start, timestamp_end) ## CONNECTION WITH API

        transformed_data = feature_engineering_pipeline(historical_data, transformation_config)

        return transformed_data

    

In [ ]:
#initializing the info manager
im = create_info_object()

#initializing the anomaly detector
ad=AnomalyDetector()

In [ ]:

while True: #loops continuosly

    #first we call get_datapoint and we wait for a new input to arrive
    new_datapoint = get_datapoint() ## CONNECTION WITH API

    #once the new data point is aquired we clean it
    cleaned_datapoint = cleaning_pipeline(new_datapoint, im)

    #we now check if some drift has been detected
    drift_flag=ADWIN_drift(cleaned_datapoint, im)

    # we need to extract the following from cleaned_datapoint
    # machine_name, asset_id, kpi, operation
    
    #we call the database to extract historical data
    historical_data = get_historical_data(machine_name, asset_id, kpi, operation, None, None) ## CONNECTION WITH API
    historical_data = historical_data.drop(columns=['anomaly'])
    #['anomaly']='Normal'
    historical_data.loc[len(get_historical_data)] = cleaned_datapoint

    if drift_flag==True:

        #retrain anomaly detection model
        model=ad.train(historical_data[:]) #Here we should put the get_historical_data()
        im.update_model_ad(cleaned_datapoint, model)

        #retrain forecasting algorithm model
        model_info = tdnn_forecasting_training(historical_data[:])  #contains [best_model_TDNN, best_params, stats] #Here we should put the get_historical_data()
        im.update_model_forecast(cleaned_datapoint, model_info)
        
    #Anomalies detection branch 
    # get de model    
    ad_model=im.get_model_ad(cleaned_datapoint)
    #predict class
    cleaned_datapoint['anomaly']=ad.predict(cleaned_datapoint , ad_model)

    if cleaned_datapoint['anomaly']=="Anomaly":
        anomaly_identity = {key: cleaned_datapoint[key] for key in identity if key in cleaned_datapoint}
        
        _ = send_alert(anomaly_identity)
    
    # MAKE cleaned_datapoint a dictionary 
    ready_to_store_datapoint = {
    'timestamp': 'timepoint',
    'isset_id': 'ast-yhccl1zjue2t',
    'name': 'metal_cutting',
    'kpi': 'time',
    'operation': 'working',
    'anomaly': str,
    'sum': float, 
    'avg': float,
    'min': float,
    'max': float,
    'var': float}
    
    _ = store_datapoint(ready_to_store_datapoint) ## CONNECTION WITH API


    

    

In [ ]:

#Feature forecasting branch
#Activation whenever there is a call
#num_predictions comes from the query
num_predictions = 7 #default
forecasting_model_info=im.get_model_forecast(current_dp)
transformed_dp = fef.feature_engineering_pipeline(current_dp, infor.ML_algorithms_config)
forecasting_model = forecasting_model[0]
forecasting_params = forecasting_model[1]
forecasting_stats = forecasting_model[2]
predictions = tdnn_forecasting_prediction(forecasting_model, forecasting_params['tau'], transformed_dp , num_predictions, forecasting_stats)
#Send prediction should be implemented

#The feature engineering will be added in a future development.